In [171]:
import nbimporter
from numpy.random import laplace, poisson, normal
from numpy import linspace, meshgrid, subtract, exp, divide, add, amax, sqrt, median, std, array
from numpy.linalg import lstsq
from model_calls import showim

In [172]:
resol = 1024.0
vignt = 3.0
probe = 1.0e+6
multis = 10

In [173]:
x, y = meshgrid(linspace(0, resol-1, int(resol)),
                linspace(0, resol-1, int(resol)))

In [174]:
kernel = subtract(1.0, exp(divide(x, -resol/vignt))) + subtract(1.0, exp(divide(y, -resol/vignt))) + \
    subtract(1.0, exp(divide(x-resol, resol/vignt))) + \
    subtract(1.0, exp(divide(y-resol, resol/vignt)))
kernel /= amax(kernel)
kernel *= probe
showim(image=kernel, fname='kernel')

In [175]:
kernels = []
for i in range(0, multis):
    noise = normal(loc=0, scale=sqrt(probe), size=(int(resol), int(resol)))
    kernels.append(kernel+noise)

In [176]:
kernelmed = median(kernels, axis=0)
showim(image=kernelmed, fname='kernelmed')

In [177]:
print median(kernelmed-kernel), std(kernelmed-kernel)

0.0899253304815 372.224972619


In [ ]:
u = x.flatten()
v = y.flatten()
zerdeg = [u*0+1]
onedeg = [u, v]
twodeg = [u**2, v**2, u*v]
thrdeg = [u**3, v**3, u**2*v, u*v**2]
foudeg = [u**4, v**4, u**3*v, u*v**3, u**2*v**2]
fivdeg = [u**5, v**5, u**4*v, u*v**4, u**3*v**2, u**2*v**3]
sixdeg = [u**6, v**6, u**5*v, u*v**5, u**4*v**2, u**2*v**4, u**3*v**3]
sevdeg = [u**7, v**7, u**6*v, u*v**6, u**5*v**2, u**2*v**5, u**4*v**3, u**3*v**4]
combin = zerdeg+onedeg+twodeg+thrdeg+foudeg+fivdeg+sixdeg+sevdeg
A = array(combin).T
B = kernelmed.flatten()
coeff, r, rank, s = lstsq(A,B, rcond=-1)

In [ ]:
zerpol = [(x*0+coeff[0])]
onepol = [x*coeff[1], y*coeff[2]]
twopol = [(x**2)*coeff[3], (y**2)*coeff[4], (x*y)*coeff[5]]
thrpol = [(x**3)*coeff[6], (y**3)*coeff[7], ((x**2)*y)*coeff[8], (x*(y**2))*coeff[9]]
foxpol = [(x**4)*coeff[10], (y**4)*coeff[11], ((x**3)*y)*coeff[12], (x*(y**3))*coeff[13], ((x**2)*(y**2))*coeff[14]]
fiypol = [(x**5)*coeff[15], (y**5)*coeff[16], ((x**4)*y)*coeff[17], (x*(y**4))*coeff[18], ((x**3)*(y**2))*coeff[19], ((x**2)*(y**3))*coeff[20]]
sixpol = [(x**6)*coeff[21], (y**6)*coeff[22], ((x**5)*y)*coeff[23], (x*(y**5))*coeff[24], ((x**4)*(y**2))*coeff[25], ((x**2)*(y**4))*coeff[26], ((x**3)*(y**3))*coeff[27]]
sevpol = [(x**7)*coeff[28], (y**7)*coeff[29], ((x**6)*y)*coeff[30], (x*(y**6))*coeff[31], ((x**5)*(y**2))*coeff[32], ((x**2)*(y**5))*coeff[33], ((x**4)*(y**3))*coeff[34], ((x**3)*(y**4))*coeff[35]]
kernelpol = zerpol+onepol+twopol+thrpol+foxpol+fiypol+sixpol+sevpol
kernelfit = sum(kernelpol)
showim(image=kernelfit, fname='kernelfit')

In [178]:
print median(kernelfit-kernel), std(kernelfit-kernel)
print median(kernelmed-kernelfit), std(kernelmed-kernelfit)